# Export Functionality Demo

This notebook demonstrates the enhanced export functionality for DIP_SMC_PSO simulations.

## Features
- Export simulation data in multiple formats (CSV, NumPy, JSON)
- Save plots in high resolution (PNG, PDF)
- Generate comprehensive analysis reports
- Organize exports with automatic timestamping

In [ ]:
# Standard imports
import numpy as np
import matplotlib.pyplot as plt
import sys
from pathlib import Path

# Add src to path for imports
project_root = Path().absolute().parent
sys.path.insert(0, str(project_root / 'src'))

# Project imports
from src.config import load_config
from controllers.factory import create_controller
from core.dynamics import DIPDynamics
from core.simulation_runner import run_simulation
from utils.visualization import Visualizer
from utils.notebook_export import NotebookExporter, quick_export_simulation, quick_analysis_report

print("✅ All imports successful!")

## 1. Run a Sample Simulation

In [ ]:
# Load configuration
config = load_config('../config.yaml')

# Create dynamics and controller
dynamics = DIPDynamics(config.dip_params)
controller = create_controller('classical_smc', config)

# Set initial conditions (small perturbation)
initial_state = np.array([0.1, 0.1, -0.1, 0.0, 0.0, 0.0])

# Run simulation
t, x, u = run_simulation(
    dynamics, controller,
    initial_state=initial_state,
    duration=5.0,
    dt=0.01
)

print(f"✅ Simulation completed: {len(t)} time steps over {t[-1]:.1f} seconds")
print(f"   Final position: {x[-1, 0]:.4f} m")
print(f"   Final angles: θ₁={np.rad2deg(x[-1, 1]):.2f}°, θ₂={np.rad2deg(x[-1, 2]):.2f}°")

## 2. Create Visualizations

In [ ]:
# Create state plots
fig1, axes = plt.subplots(3, 1, figsize=(12, 8))

# Position
axes[0].plot(t, x[:, 0], 'b-', linewidth=2)
axes[0].set_ylabel('Position (m)')
axes[0].grid(True, alpha=0.3)
axes[0].set_title('Classical SMC Control Performance')

# Angles
axes[1].plot(t, np.rad2deg(x[:, 1]), 'r-', linewidth=2, label='θ₁')
axes[1].plot(t, np.rad2deg(x[:, 2]), 'g-', linewidth=2, label='θ₂')
axes[1].set_ylabel('Angle (degrees)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Control input
axes[2].plot(t[:-1], u, 'k-', linewidth=2)
axes[2].set_ylabel('Control Force (N)')
axes[2].set_xlabel('Time (s)')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Create phase space plot
fig2, axes = plt.subplots(1, 2, figsize=(12, 5))

# Link 1 phase space
axes[0].plot(np.rad2deg(x[:, 1]), np.rad2deg(x[:, 4]), 'r-', alpha=0.7, linewidth=2)
axes[0].set_xlabel('θ₁ (degrees)')
axes[0].set_ylabel('θ̇₁ (degrees/s)')
axes[0].set_title('Link 1 Phase Space')
axes[0].grid(True, alpha=0.3)

# Link 2 phase space
axes[1].plot(np.rad2deg(x[:, 2]), np.rad2deg(x[:, 5]), 'g-', alpha=0.7, linewidth=2)
axes[1].set_xlabel('θ₂ (degrees)')
axes[1].set_ylabel('θ̇₂ (degrees/s)')
axes[1].set_title('Link 2 Phase Space')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Export Data and Plots

In [ ]:
# Initialize exporter
exporter = NotebookExporter(base_dir="../exports")

# Export simulation data with metadata
metadata = {
    "controller_type": "classical_smc",
    "gains": controller.gains.tolist(),
    "initial_conditions": initial_state.tolist(),
    "simulation_duration": float(t[-1]),
    "timestep": float(t[1] - t[0]),
    "dynamics_model": "simplified"
}

data_zip = exporter.export_simulation_data(
    time=t, 
    states=x, 
    controls=u, 
    metadata=metadata,
    prefix="classical_smc_demo"
)

In [ ]:
# Export plots in multiple formats
plot_dir = exporter.export_plots(
    figures=[fig1, fig2],
    names=["time_series", "phase_space"],
    prefix="classical_smc_plots",
    formats=["png", "pdf"]
)

## 4. Generate Comprehensive Analysis Report

In [ ]:
# Create a comprehensive analysis report
report_path = exporter.create_analysis_report(
    time=t,
    states=x, 
    controls=u,
    metadata=metadata,
    prefix="classical_smc_analysis"
)

print(f"📊 Open the report: {report_path}")

## 5. Quick Export Functions

For convenience, you can also use the quick export functions:

In [ ]:
# Quick data export
quick_data_zip = quick_export_simulation(
    t, x, u, 
    metadata={"method": "quick_export"}, 
    prefix="quick_demo"
)

# Quick analysis report
quick_report = quick_analysis_report(
    t, x, u,
    metadata={"method": "quick_analysis"},
    prefix="quick_analysis"
)

## 6. Advanced Visualization Export

Demonstrate the enhanced Visualizer class with export capabilities:

In [ ]:
# Create enhanced visualizations
vis = Visualizer(dynamics)

# Create static snapshot
vis.save_static_plot(
    state_history=x,
    filename="../exports/final_state_snapshot.png",
    time_step=-1,  # Final state
    dpi=300
)

# Create phase plot
phase_fig = vis.create_phase_plot(
    state_history=x,
    filename="../exports/enhanced_phase_plot.png",
    dpi=300
)

# Display the phase plot
plt.show()

## 7. Animation Export (Optional)

Create and save an animation of the pendulum motion:

In [ ]:
# Create animation (subsample for faster rendering)
subsample = slice(None, None, 5)  # Every 5th frame
ani = vis.animate(
    time_history=t[subsample],
    state_history=x[subsample],
    control_history=u[subsample[:-1]] if len(u) == len(t)-1 else u[subsample],
    dt=0.05  # 20 FPS
)

# Save as MP4 (requires ffmpeg)
try:
    vis.save_animation("../exports/pendulum_animation.mp4", fps=20, bitrate=1800)
except Exception as e:
    print(f"⚠️  Animation export failed (ffmpeg might not be installed): {e}")
    print("💡 Install ffmpeg to enable MP4 export")

# Save as GIF (more compatible)
try:
    vis.save_animation("../exports/pendulum_animation.gif", fps=10)
    print("✅ Animation saved as GIF")
except Exception as e:
    print(f"⚠️  GIF export failed: {e}")

## Summary

This notebook demonstrated:

1. **Data Export**: Simulation results in CSV, NumPy, and JSON formats
2. **Plot Export**: High-resolution plots in PNG and PDF formats
3. **Analysis Reports**: Comprehensive HTML reports with statistics and visualizations
4. **Animation Export**: MP4 and GIF animations of pendulum motion
5. **Quick Functions**: Convenient one-line export functions
6. **Enhanced Visualizations**: Static snapshots and phase space plots

All exports are automatically organized with timestamps and can be easily shared or used for further analysis.

### Files Created:
- `exports/classical_smc_demo_YYYYMMDD_HHMMSS.zip` - Complete simulation data
- `exports/classical_smc_plots_YYYYMMDD_HHMMSS/` - Plot files
- `exports/classical_smc_analysis_report_YYYYMMDD_HHMMSS.html` - Analysis report
- `exports/final_state_snapshot.png` - Static pendulum visualization
- `exports/enhanced_phase_plot.png` - Phase space analysis
- `exports/pendulum_animation.mp4` or `.gif` - Animation (if supported)